In [1]:
import os
import pandas as pd

In [2]:
## Merge two dates

In [6]:
DATA_DIR = os.getenv("DATA_DIR")
bq = os.path.join(DATA_DIR, "bq_journey_extract")
files = [os.path.join(bq, file) for file in os.listdir(bq) if "pageseq_user_journey_brexit" in file
and "doo" not in file]

In [10]:
df = pd.concat([pd.read_csv(f, 
                 compression="gzip", 
                 sep="\t", 
                 usecols=['Occurrences', 'PageSeq_Length', 'PageSequence']) for f in files])
df.head()

,Occurrences,PageSeq_Length,PageSequence
0,6813,3,/prepare-eu-exit>>/prepare-eu-exit/going-and-b...
1,6721,2,/guidance/prepare-to-drive-in-the-eu-after-bre...
2,6507,2,/guidance/passport-rules-for-travel-to-europe-...
3,5680,2,/guidance/uk-nationals-travelling-to-eu-essent...
4,4543,2,/settled-status-eu-citizens-families/applying-...


In [12]:
df.shape

(1756235, 3)

In [14]:
# ids = df["PageSequence"]
# df[ids.isin(ids[ids.duplicated()])].sort_values("PageSequence")

In [16]:
df['Occurrences'] = df.groupby("PageSequence")['Occurrences'].transform(sum)

In [18]:
df.drop_duplicates("PageSequence", inplace=True)
df.shape

(1699292, 3)

In [19]:
df[df['Occurrences']==1].shape

(1575222, 3)

In [20]:
pagelist = [pageseq.split(">>") for pageseq in df['PageSequence'].values]
df['Page_List'] = pagelist

In [22]:
df.drop("PageSeq_Length", axis=1, inplace=True)

In [23]:
## Save

In [25]:
bq_file = os.path.join(bq, "pageseq_user_journey_brexit_feb19_mar_30.csv.gz")
df.to_csv(bq_file, compression="gzip", sep='\t', index=False)

In [30]:
# Merge edgelists

In [96]:
net = os.path.join(DATA_DIR, "processed_network")
brexit_edges_file = os.path.join(net, "feb19_mar30_nos_edges.csv.gz")

In [97]:
dfb = pd.read_csv(brexit_edges_file, sep="\t", compression="gzip")

In [98]:
dfb = dfb[~(dfb.Source_node.str.contains(" ")) & ~(dfb.Destination_node.str.contains(" "))]

In [99]:
dfb.head()

,Source_node,Source_id,Destination_node,Destination_id,Weight
0,/prepare-eu-exit,0,/prepare-eu-exit/going-and-being-abroad,1,102737
1,/prepare-eu-exit/going-and-being-abroad,1,/guidance/uk-nationals-travelling-to-eu-essent...,2,39123
2,/guidance/prepare-to-drive-in-the-eu-after-brexit,3,/guidance/international-driving-permits-for-uk...,4,66923
3,/guidance/passport-rules-for-travel-to-europe-...,5,/guidance/passport-rules-for-travel-to-europe-...,5,62934
4,/guidance/uk-nationals-travelling-to-eu-essent...,2,/guidance/uk-nationals-travelling-to-eu-essent...,2,29876


In [100]:
dfb.shape

(1115449, 5)

In [101]:
dfb[dfb.Weight!=1].shape

(334812, 5)

In [102]:
dfb = dfb[dfb.Weight!=1]

In [103]:
### Directories
train_network = os.path.join(DATA_DIR, "train_network")
edgefile = os.path.join(train_network, "merged_edges_struct_funct.csv.gz")
edges = pd.read_csv(edgefile, compression="gzip")

In [104]:
edges.head()

,dest_cid,dest_node,source,src_cid,src_node,target,weight,edge-id
0,1e333395-5dd5-4452-96a3-fbe939928761,/visa-fees,33265,5ef7560d-7631-11e4-a3cb-005056011aef,/government/publications/guidance-for-dependan...,58314,66.0,33265-58314
1,aa055bd6-fde1-4471-8ac2-ac86241b4898,/find-a-visa-application-centre,58314,1e333395-5dd5-4452-96a3-fbe939928761,/visa-fees,12343,1164.0,58314-12343
2,305acc88-488e-44ec-8e5a-1a5de7819ba9,/uk-family-visa,23720,d612c61e-22f4-4922-8bb2-b04b9202126e,/entering-staying-uk/family-visas,25053,377.0,23720-25053
3,305acc88-488e-44ec-8e5a-1a5de7819ba9,/uk-family-visa/partner-spouse,25053,305acc88-488e-44ec-8e5a-1a5de7819ba9,/uk-family-visa,14044,26227.0,25053-14044
4,5ef421be-7631-11e4-a3cb-005056011aef,/government/publications/application-for-uk-vi...,14044,305acc88-488e-44ec-8e5a-1a5de7819ba9,/uk-family-visa/partner-spouse,33797,2733.0,14044-33797


In [105]:
nodes_b = set(list(dfb.Source_node.values) + list(dfb.Destination_node.values))

In [106]:
len(nodes_b)

65194

In [107]:
nodes = set(list(edges.src_node.values) + list(edges.dest_node.values))

In [108]:
len(nodes.difference(nodes_b)), len(nodes_b.difference(nodes))

(79302, 36632)

In [109]:
def build_dict(k1,v1,k2,v2):
    agg_dict = dict(zip(k1,v1))
    for key,value in zip(k2,v2):
        if key not in agg_dict.keys():
            agg_dict[key] = value
    return agg_dict   

In [110]:
nid_cid = build_dict(edges.source, edges.src_cid, edges.target, edges.dest_cid)
nid_url = build_dict(edges.source, edges.src_node, edges.target, edges.dest_node)
url_nid = build_dict(edges.src_node, edges.source, edges.dest_node, edges.target)

In [111]:
max_nid = max(url_nid.values()) +1
max_nid

107864

In [112]:
for node in nodes_b:
    if node not in url_nid.keys():
        url_nid[node] = max_nid
        max_nid +=1

In [113]:
max(url_nid.values())

144495

In [114]:
dfb['source'] = dfb.Source_node.map(url_nid)
dfb['target'] = dfb.Destination_node.map(url_nid)

In [117]:
dfb.columns

Index(['Source_node', 'Destination_node', 'Weight', 'source', 'target'], dtype='object')

In [116]:
dfb.drop(['Source_id', 'Destination_id'], axis=1, inplace=True)

In [120]:
dfb.rename(columns={'Source_node':'src_node',
                    'Destination_node':'dest_node',
                    'Weight':'weight'}, inplace=True)

In [121]:
dfb['src_cid'] = dfb.source.map(nid_cid)
dfb['dest_cid'] = dfb.target.map(nid_cid)

In [126]:
edges.columns

Index(['dest_cid', 'dest_node', 'source', 'src_cid', 'src_node', 'target',
       'weight', 'edge-id'],
      dtype='object')

## Save out relevant stuff

In [131]:
dfb['edge-id'] = dfb[['source', 'target']].apply(lambda x: "{}-{}".format(x[0],x[1]), axis=1)

In [137]:
outs = dfb[~dfb['edge-id'].isin(edges['edge-id'].values)]

In [139]:
outs.shape, dfb.shape

((242200, 8), (334812, 8))

In [147]:
merged_edgefile = os.path.join(train_network, "merged_edges_struct_funct_w_brexit.csv.gz")

In [148]:
merged = pd.concat([outs,edges], sort=True, ignore_index=True)

In [151]:
merged.shape, edges.shape, outs.shape

((505956, 8), (263756, 8), (242200, 8))

In [153]:
merged.to_csv(merged_edgefile, compression='gzip', sep='\t', index=False)

## Save out only brexit edges 
not included in original edgelist that was used to train n2viteration 1

In [145]:
outs_edgefile = os.path.join(train_network, "funct_only_brexit_1902_3003.csv.gz")

In [154]:
outs.to_csv(outs_edgefile, compression='gzip', sep='\t', index=False)